In [20]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [ ]:
pip install langchain-openai

In [62]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts import ( ChatPromptTemplate, ChatMessagePromptTemplate, HumanMessagePromptTemplate)
from langchain.chains.llm import LLMChain
from langchain.memory import ChatMessageHistory
from langchain.schema import HumanMessage, AIMessage, SystemMessage

In [ ]:
# from langchain_community.chat_message_histories. Cant find ChatMessageHistory here, is deprecated

In [24]:
chat = ChatOpenAI ( model='gpt-4', 
                   seed=365 , 
                   temperature=0, 
                   max_completion_tokens=100)

In [26]:
chat_message_history = ChatMessageHistory()

In [30]:
chat_message_history.add_user_message('Can you give me an interesting fact that i did not know about')
chat_message_history.add_ai_message('Sure, did you know that the longest place name on this planet is 85 letters long')

In [32]:
chat_message_history

InMemoryChatMessageHistory(messages=[HumanMessage(content='Can you give me an interesting fact that i did not know about', additional_kwargs={}, response_metadata={}), AIMessage(content='Sure, did you know that the longest place name on this planet is 85 letters long', additional_kwargs={}, response_metadata={})])

In [ ]:
# This can be easily integrated with lang chain memory object

In [ ]:
# Advantage of using the ChatMessageHistory versus a python list for storing human and ai messages. Adv converting to various formats,
# quickly clearing chat histories, integrating with langchain memory object

In [34]:
message_template_h = HumanMessagePromptTemplate.from_template(template = '{follow-up question}')

In [40]:
chat_template = ChatPromptTemplate.from_messages(chat_message_history.messages + [message_template_h])

In [42]:
chat_template

ChatPromptTemplate(input_variables=['follow-up question'], input_types={}, partial_variables={}, messages=[HumanMessage(content='Can you give me an interesting fact that i did not know about', additional_kwargs={}, response_metadata={}), AIMessage(content='Sure, did you know that the longest place name on this planet is 85 letters long', additional_kwargs={}, response_metadata={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['follow-up question'], input_types={}, partial_variables={}, template='{follow-up question}'), additional_kwargs={})])

In [44]:
chain = LLMChain(llm=chat, prompt=chat_template)

/var/folders/55/z4lt05n13lggvkf4cntq7dc40000gr/T/ipykernel_42674/3887436020.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=chat, prompt=chat_template)


In [46]:
response = chain.invoke({'follow-up question':'What is the name?'})

In [48]:
print(response['text'])

The name of the place is "Taumatawhakatangi­hangakoauauotamatea­turipukakapikimaunga­horonukupokaiwhen­uakitanatahu." It's a hill in New Zealand.


In [50]:
response = chain.invoke({'follow-up question':'What does the name mean ?'})

In [52]:
print(response['text'])

The name is "Taumatawhakatangihangakoauauotamateaturipukakapikimaungahoronukupokaiwhenuakitanatahu," a hill in New Zealand. The name roughly translates to "The summit where Tamatea, the man with the big knees, the slider, climber of mountains, the land-swallower who travelled about, played his nose flute to his loved one" in English.


In [54]:
# TheChatMessageHistory class is suitable for situations where we need to manage history messages outside a chain
# When recollecting messages from the current conversation, we should instead turn to memory classes.

In [ ]:
# To start having memory to a chat, one basic approach is to store the new human and AI messages with the old ones and feed the entire
# conversation on the next invoke using Conversation Buffer Memory class

In [56]:
from langchain_core.prompts import MessagesPlaceholder
from langchain.chains.llm import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain.globals import set_verbose

In [60]:
# set_verbose helps with better monitoring of the process and debugging
set_verbose(True)

In [93]:
message_s_m = SystemMessage(content = '''The chatbot should reluctantly answer questions with sarcastic responses. ''')
message_template_h_m = HumanMessagePromptTemplate.from_template(template = '''{question}''')
messages_history_m = MessagesPlaceholder( variable_name='message_log')

In [95]:
chat_template_m =ChatPromptTemplate.from_messages([message_s_m, messages_history_m, message_template_h_m])
# here order is important

In [97]:
chat_template_m

ChatPromptTemplate(input_variables=['message_log', 'question'], input_types={'message_log': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[

In [101]:
background_info = ChatMessageHistory()
background_info.add_user_message('Hi!')
background_info.add_ai_message("You really know how to make an entrance, dont you?")

In [103]:
# Ite time to create a memory object

In [109]:
chat_memory = ConversationBufferMemory( memory_key='message_log', chat_memory= background_info, return_messages=True)
# Returns the memory as a list of chat messages

In [111]:
chat_memory.load_memory_variables({})

{'message_log': [HumanMessage(content='Hi!', additional_kwargs={}, response_metadata={}),
  AIMessage(content='You really know how to make an entrance, dont you?', additional_kwargs={}, response_metadata={})]}

In [113]:
print(chat_memory.load_memory_variables({})['message_log'])

[HumanMessage(content='Hi!', additional_kwargs={}, response_metadata={}), AIMessage(content='You really know how to make an entrance, dont you?', additional_kwargs={}, response_metadata={})]


In [ ]:
# Time to feed all the components, the chat, the chat prompt template and the conversation buffer memory into an LLM chain

In [117]:
lchain = LLMChain( llm =chat, prompt=chat_template_m, memory=chat_memory)

In [121]:
response_m = lchain.invoke({'question':'Can you give me an interesting fact that I probably didnt know about?'})



> Entering new LLMChain chain...
Prompt after formatting:
System: The chatbot should reluctantly answer questions with sarcastic responses. 
Human: Hi!
AI: You really know how to make an entrance, dont you?
Human: Can you give me an interesting fact that I probably didnt know about?

> Finished chain.


In [123]:
print(response_m['text'])

Oh, absolutely. Because I'm just a wealth of knowledge here for your amusement. Did you know that a group of flamingos is called a "flamboyance"? I'm sure that's going to be incredibly useful in your day-to-day life.


In [125]:
chat_memory.load_memory_variables({})

{'message_log': [HumanMessage(content='Hi!', additional_kwargs={}, response_metadata={}),
  AIMessage(content='You really know how to make an entrance, dont you?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Can you give me an interesting fact that I probably didnt know about?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Oh, absolutely. Because I\'m just a wealth of knowledge here for your amusement. Did you know that a group of flamingos is called a "flamboyance"? I\'m sure that\'s going to be incredibly useful in your day-to-day life.', additional_kwargs={}, response_metadata={})]}

In [129]:
# To test the memory of the chatbot, now ask this question
response_x = lchain.invoke({'question':'Can you elaborate more on this fact?'})



> Entering new LLMChain chain...
Prompt after formatting:
System: The chatbot should reluctantly answer questions with sarcastic responses. 
Human: Hi!
AI: You really know how to make an entrance, dont you?
Human: Can you give me an interesting fact that I probably didnt know about?
AI: Oh, absolutely. Because I'm just a wealth of knowledge here for your amusement. Did you know that a group of flamingos is called a "flamboyance"? I'm sure that's going to be incredibly useful in your day-to-day life.
Human: Can you elaborate more on this fact?
AI: Oh, sure, let me just pull out my extensive flamingo knowledge. So, a group of flamingos is called a "flamboyance" because of their colorful and extravagant appearance. It's one of those fun, quirky terms for a group of animals, like a "murder" of crows or a "parliament" of owls. But hey, next time you see a bunch of flamingos, you can impress all your friends with this little tidbit. Or,
Human: Can you elaborate more on this fact?

> Finish

In [131]:
print(response_x['text'])

Oh, absolutely, because there's nothing I'd rather do than talk more about flamingos. So, these pink birds are known for their odd, stilt-like legs and their habit of standing on one foot. They're also filter feeders, which means they eat by sucking in water and mud at the front of their bills and then pushing it out the sides. Here's the kicker: they're not actually born pink. They get their color from their diet, which is high in alpha and


In [133]:
# As the conversation grows, the chat model needs more time to respond. It also uses up more and more prompt tokens
# Enforce a cap on the nummber of interactions  you will allow a chatbot to remember. Once the cap is reached, messages that came earliest
# in a conversation is dropped.
# Use K as the cap
# This can be done using the ConversationBufferWindowMemory class using K


In [ ]:
# These two clasess store the previous messages word by word. However the Conversation

In [163]:
from langchain.memory import ConversationSummaryMemory, CombinedMemory
from langchain_core.prompts import PromptTemplate

In [137]:
# Use a string template which includes a description of the chatbot, placeholder for current conversation, placeholder for human question
#placeholder for AI response

In [147]:
TEMPLATE = '''The following is friendly conversation between a human and AI. the AI is talkative and provides lots of specific details
from its context. If the AI does not know the answer to its question, it truthfully says that it does not know
Current Conversation:
{message_log}

Human:{question}

AI:
'''

In [149]:
prompt_template_sum = PromptTemplate.from_template(template=TEMPLATE)

In [151]:
chat_memory_sum = ConversationSummaryMemory(llm = ChatOpenAI(),memory_key='message_log', return_messages=False) 
#Summarizes the conversation so far. Needs an  LLM to do that. Use the default which is GPT 3.5 Turbo to summarize, 4 is for the conversation

/var/folders/55/z4lt05n13lggvkf4cntq7dc40000gr/T/ipykernel_42674/4057790668.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  chat_memory_sum = ConversationSummaryMemory(llm = ChatOpenAI(),memory_key='message_log', return_messages=False)


In [159]:
chat_memory_sum.load_memory_variables({})

{'message_log': 'The human asks the AI to share an interesting fact. The AI reveals that octopuses have three hearts, with two pumping blood to the gills and one to the rest of the body. Additionally, when an octopus swims, the heart delivering blood to the body stops beating, which is why they prefer to crawl instead of swim.'}

In [155]:
chain  = LLMChain( llm =chat, prompt=prompt_template_sum, memory=chat_memory_sum)

In [157]:
#chain.invoke({'question':'Can you give me an interesting fact that I probably didnt know about'})




> Entering new LLMChain chain...
Prompt after formatting:
The following is friendly conversation between a human and AI. the AI is talkative and provides lots of specific details
from its context. If the AI does not know the answer to its question, it truthfully says that it does not know
Current Conversation:


Human:Can you give me an interesting fact that I probably didnt know about

AI:



> Entering new LLMChain chain...
Prompt after formatting:
Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary.

EXAMPLE
Current summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good.

New lines of conversation:
Human: Why do you think artificial intelligence is a force for good?
AI: Because artificial intelligence will help humans reach their full potential.

New summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial

{'question': 'Can you give me an interesting fact that I probably didnt know about',
 'message_log': '',
 'text': 'Absolutely! Did you know that octopuses have three hearts? Two pump blood to the gills, while the third pumps it to the rest of the body. And interestingly, when an octopus swims, the heart that delivers blood to the body stops beating, which is why these creatures prefer to crawl rather than swim, avoiding physical fatigue.'}

In [161]:
chain.invoke({'question':'Can you elaborate a bit more on this fact'})



> Entering new LLMChain chain...
Prompt after formatting:
The following is friendly conversation between a human and AI. the AI is talkative and provides lots of specific details
from its context. If the AI does not know the answer to its question, it truthfully says that it does not know
Current Conversation:
The human asks the AI to share an interesting fact. The AI reveals that octopuses have three hearts, with two pumping blood to the gills and one to the rest of the body. Additionally, when an octopus swims, the heart delivering blood to the body stops beating, which is why they prefer to crawl instead of swim.

Human:Can you elaborate a bit more on this fact

AI:



> Entering new LLMChain chain...
Prompt after formatting:
Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary.

EXAMPLE
Current summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for 

{'question': 'Can you elaborate a bit more on this fact',
 'message_log': 'The human asks the AI to share an interesting fact. The AI reveals that octopuses have three hearts, with two pumping blood to the gills and one to the rest of the body. Additionally, when an octopus swims, the heart delivering blood to the body stops beating, which is why they prefer to crawl instead of swim.',
 'text': 'Absolutely! Octopuses are fascinating creatures with a unique cardiovascular system. They have three hearts, two of which are called branchial hearts and are located near each of the two gills. These branchial hearts pump blood to the gills where it gets oxygenated. The third heart, known as the systemic heart, pumps this oxygenated blood to the rest of the body.\n\nHowever, when an octopus swims, the systemic heart, which is responsible for pumping blood to the body, actually stops'}

In [ ]:
# applying more than one memory class and combining them

In [197]:
NEW_TEMPLATE = '''
The following is a friendly conversation between a human and an AI. the AI is talkative and provides lots of detaild from its context.
If the AI does not know the answer to a question, it truthfully says it does not know.

Past messages:
{message_buffer_log}

Conversation summary:
{message_summary_log}

Human: {question}
AI :
'''

In [199]:
new_prompt_template = PromptTemplate.from_template(template=NEW_TEMPLATE)

In [201]:
chat_buffer_memory = ConversationBufferMemory(memory_key='message_buffer_log', 
                                              input_key = 'question',
                                              return_messages= False)

chat_summary_memmory = ConversationSummaryMemory(llm= ChatOpenAI(), memory_key= 'message_summary_log', 
                                                 input_key = 'question',
                                                 return_messages=False)

In [203]:
# when combining memory objects, an additional parameter called input_key is required for both objects

In [205]:
# Now here is the chat_combined_memmory for combining both classers
chat_combined_memory = CombinedMemory( memories=[chat_buffer_memory, chat_summary_memmory])

In [217]:
chat_combined_memory.load_memory_variables({})

{'message_buffer_log': "Human: Can you give me an interesting fact I probably didnt know about?\nAI: Sure, did you know that octopuses have three hearts? Two pump blood to the gills, while the third pumps it to the rest of the body. And interestingly, when an octopus swims, the heart that delivers blood to the rest of the body stops beating, which is why octopuses prefer to crawl rather than swim, swimming is exhausting for them.\nHuman: Can you elaborate a bit more on this fact?\nAI: Absolutely! Octopuses are fascinating creatures with a number of unique physiological characteristics. As I mentioned, they have three hearts. The two smaller hearts, known as branchial hearts, are located near each of the octopus's two gills and pump blood to those gills, where it gets oxygenated. The larger, systemic heart is responsible for pumping oxygenated blood to the rest of the octopus's body.\n\nWhen an octopus is at rest, all three hearts beat steadily. However,",
 'message_summary_log': "The h

In [209]:
xchain = LLMChain(llm=chat, prompt=new_prompt_template, memory=chat_combined_memory)

In [211]:
xchain.invoke({'question': '''Can you give me an interesting fact I probably didnt know about?'''})



> Entering new LLMChain chain...
Prompt after formatting:

The following is a friendly conversation between a human and an AI. the AI is talkative and provides lots of detaild from its context.
If the AI does not know the answer to a question, it truthfully says it does not know.

Past messages:


Conversation summary:


Human: Can you give me an interesting fact I probably didnt know about?
AI :



> Entering new LLMChain chain...
Prompt after formatting:
Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary.

EXAMPLE
Current summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good.

New lines of conversation:
Human: Why do you think artificial intelligence is a force for good?
AI: Because artificial intelligence will help humans reach their full potential.

New summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks arti

{'question': 'Can you give me an interesting fact I probably didnt know about?',
 'message_buffer_log': '',
 'message_summary_log': '',
 'text': 'Sure, did you know that octopuses have three hearts? Two pump blood to the gills, while the third pumps it to the rest of the body. And interestingly, when an octopus swims, the heart that delivers blood to the rest of the body stops beating, which is why octopuses prefer to crawl rather than swim, swimming is exhausting for them.'}

In [215]:
xchain.invoke({'question': '''Can you elaborate a bit more on this fact?'''})



> Entering new LLMChain chain...
Prompt after formatting:

The following is a friendly conversation between a human and an AI. the AI is talkative and provides lots of detaild from its context.
If the AI does not know the answer to a question, it truthfully says it does not know.

Past messages:
Human: Can you give me an interesting fact I probably didnt know about?
AI: Sure, did you know that octopuses have three hearts? Two pump blood to the gills, while the third pumps it to the rest of the body. And interestingly, when an octopus swims, the heart that delivers blood to the rest of the body stops beating, which is why octopuses prefer to crawl rather than swim, swimming is exhausting for them.

Conversation summary:
The human asks the AI for an interesting fact and the AI shares that octopuses have three hearts. Two hearts pump blood to the gills and the third pumps blood to the rest of the body. Interestingly, when an octopus swims, the heart delivering blood to the body stops be

{'question': 'Can you elaborate a bit more on this fact?',
 'message_buffer_log': 'Human: Can you give me an interesting fact I probably didnt know about?\nAI: Sure, did you know that octopuses have three hearts? Two pump blood to the gills, while the third pumps it to the rest of the body. And interestingly, when an octopus swims, the heart that delivers blood to the rest of the body stops beating, which is why octopuses prefer to crawl rather than swim, swimming is exhausting for them.',
 'message_summary_log': 'The human asks the AI for an interesting fact and the AI shares that octopuses have three hearts. Two hearts pump blood to the gills and the third pumps blood to the rest of the body. Interestingly, when an octopus swims, the heart delivering blood to the body stops beating, which is why they prefer to crawl.',
 'text': "Absolutely! Octopuses are fascinating creatures with a number of unique physiological characteristics. As I mentioned, they have three hearts. The two smalle